### Verificar que datasets i tot està al notebook

Això al començament

In [ ]:
'''
import os, sys
print("Python executable:", sys.executable)     # mostra el Python que corre (ha de ser el env activat)
print("Directori de treball:", os.getcwd())    # carpeta del repo
print("Llistat d'arxius a la carpeta:")
for f in sorted(os.listdir('.')):
    print(" -", f)
'''

## Apartat de generar xarxes amb error

 Veure d'on sortien els errors q feia servir Teresa

In [ ]:
'''
L = np.load("A2_blueprint.npy")
Nx = L.shape[0]
edges = int(L.sum())
zeros = Nx*Nx - edges
e_error = 327
z_error = 327
q_equiv = e_error / edges
p_equiv = z_error / zeros
print("edges", edges, "zeros", zeros)
print("q_equiv =", q_equiv)
print("p_equiv =", p_equiv)
'''

### Càlcul de nombre d'errors equivalents per p i q

In [ ]:
'''
# Carrega el blueprint A2 desat
L = np.load("A2_blueprint.npy")

Nx = L.shape[0]

print("Blueprint A2 carregat correctament.")
print("Mida:", L.shape)
print("Nombre d'enllaços (1’s):", int(L.sum()))
'''

'\n# Carrega el blueprint A2 desat\nL = np.load("A2_blueprint.npy")\n\nNx = L.shape[0]\n\nprint("Blueprint A2 carregat correctament.")\nprint("Mida:", L.shape)\nprint("Nombre d\'enllaços (1’s):", int(L.sum()))\n'

In [ ]:
'''# Errors deduits dels valors del paper
edges_A2 = L.sum()
zeros_A2 = Nx*Nx - edges_A2

q_paper = 0.15
p_paper = 0.007

errors_esborrar = int(q_paper * edges_A2)
errors_crear = int(p_paper * zeros_A2)

print(errors_esborrar, errors_crear)
'''


In [ ]:
'''# Errors deduits dels valors del paper
edges_A2 = L.sum()
zeros_A2 = Nx*Nx - edges_A2

errors_esborrar_teresa = int(q_equiv * edges_A2)
errors_crear_teresa = int(p_equiv * zeros_A2)

print(errors_esborrar_teresa, errors_crear_teresa)
'''

'# Errors deduits dels valors del paper\nedges_A2 = L.sum()\nzeros_A2 = Nx*Nx - edges_A2\n\nerrors_esborrar_teresa = int(q_equiv * edges_A2)\nerrors_crear_teresa = int(p_equiv * zeros_A2)\n\nprint(errors_esborrar_teresa, errors_crear_teresa)\n'

# Generar xarxes amb nombre d'errors

In [ ]:
'''# === Generar xarxes amb errors exactes (estil del repo) ===

K = 2             # nombre de xarxes a generar
e_error = 327       # errors esborrant enllaços
z_error = 327       # errors creant enllaços

# Carregar blueprint si cal
if 'L' not in globals():
    print("No hi ha L a la memòria, carregant A2_blueprint.npy ...")
    L = np.load("A2_blueprint.npy")

Nx = L.shape[0]
print("Blueprint A2 carregat: shape", L.shape, "edges:", int(L.sum()))

# Preparar output
A_repo = np.zeros((K, Nx, Nx), dtype=int)
rng = np.random.default_rng(42)  # llavor fixa

edges_total = int(L.sum())
zeros_total = Nx * Nx - edges_total

print("Edges A2:", edges_total, "Zeros A2:", zeros_total)

# Comprovacions
if e_error > edges_total:
    raise ValueError(f"e_error ({e_error}) > nombre d'enllaços ({edges_total})")
if z_error > zeros_total:
    raise ValueError(f"z_error ({z_error}) > nombre de zeros ({zeros_total})")

for m in range(K):
    A = L.copy().astype(int)
    A_flat = A.reshape(Nx*Nx)

    ones_idx = np.where(A_flat == 1)[0]
    zeros_idx = np.where(A_flat == 0)[0]

    flip_ones = rng.choice(ones_idx, size=e_error, replace=False)
    flip_zeros = rng.choice(zeros_idx, size=z_error, replace=False)

    A_flat[flip_ones] = 0
    A_flat[flip_zeros] = 1

    A = A_flat.reshape(Nx, Nx)
    A_repo[m] = A

    errors_removed = edges_total - np.sum((A == 1) & (L == 1))
    errors_added = np.sum((A == 1) & (L == 0))

    print(f"Xarxa {m}: Edges = {int(A.sum())}, Errors esborrant = {errors_removed}, Errors creant = {errors_added}")

# Guardar a disc
out_name = "synthetic_repo_errors.npy"
np.save(out_name, A_repo)
print("\nGuardades les xarxes amb errors exactes a:", out_name)
'''

'# === Generar xarxes amb errors exactes (estil del repo) ===\n\nK = 2             # nombre de xarxes a generar\ne_error = 327       # errors esborrant enllaços\nz_error = 327       # errors creant enllaços\n\n# Carregar blueprint si cal\nif \'L\' not in globals():\n    print("No hi ha L a la memòria, carregant A2_blueprint.npy ...")\n    L = np.load("A2_blueprint.npy")\n\nNx = L.shape[0]\nprint("Blueprint A2 carregat: shape", L.shape, "edges:", int(L.sum()))\n\n# Preparar output\nA_repo = np.zeros((K, Nx, Nx), dtype=int)\nrng = np.random.default_rng(42)  # llavor fixa\n\nedges_total = int(L.sum())\nzeros_total = Nx * Nx - edges_total\n\nprint("Edges A2:", edges_total, "Zeros A2:", zeros_total)\n\n# Comprovacions\nif e_error > edges_total:\n    raise ValueError(f"e_error ({e_error}) > nombre d\'enllaços ({edges_total})")\nif z_error > zeros_total:\n    raise ValueError(f"z_error ({z_error}) > nombre de zeros ({zeros_total})")\n\nfor m in range(K):\n    A = L.copy().astype(int)\n    A_f